## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [2]:
NAME = "Antonios Keremidis"
AEM = "9717"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [3]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [4]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

In [5]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [ ]:
# BEGIN CODE HERE
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import KFold, cross_validate

cls1 = LogisticRegression(random_state=RANDOM_STATE, max_iter=200,
                          solver="lbfgs", multi_class="multinomial")  # Classifier #1
cls2 = KNeighborsClassifier()  # Classifier #2
cls3 = DecisionTreeClassifier(random_state=RANDOM_STATE)  # Classifier #3

soft_vcls = VotingClassifier(estimators=[('lr', cls1), ('kn', cls2), ('dt', cls3)],
                             voting='soft', n_jobs=-1)  # Soft Voting Classifier
hard_vcls = VotingClassifier(estimators=[('lr', cls1), ('kn', cls2), ('dt', cls3)],
                             voting='hard', n_jobs=-1)  # Hard Voting Classifier

scoring = {'accuracy': make_scorer(accuracy_score),
           'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

svlcs_scores = cross_validate(
    soft_vcls, X, y, cv=kfold, scoring=scoring, n_jobs=-1)

s_avg_fmeasure = np.mean(svlcs_scores['test_f1_score'])
s_avg_accuracy = np.mean(svlcs_scores['test_accuracy'])

hvlcs_scores = cross_validate(
    hard_vcls, X, y, cv=kfold, scoring=scoring, n_jobs=-1)

h_avg_fmeasure = np.mean(hvlcs_scores['test_f1_score'])
h_avg_accuracy = np.mean(hvlcs_scores['test_accuracy'])
# END CODE HERE


In [21]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=200,
                                                 multi_class='multinomial',
                                                 random_state=42)),
                             ('kn', KNeighborsClassifier()),
                             ('dt', DecisionTreeClassifier(random_state=42))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.853 & Balanced Accuracy: 0.8258


You should achive above 82% (Soft Voting Classifier)

In [22]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=200,
                                                 multi_class='multinomial',
                                                 random_state=42)),
                             ('kn', KNeighborsClassifier()),
                             ('dt', DecisionTreeClassifier(random_state=42))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8529 & Balanced Accuracy: 0.8265


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [9]:
# BEGIN CODE HERE
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

cls1 = MLPClassifier(activation="relu", max_iter=2000,
                     random_state=RANDOM_STATE)
cls2 = GradientBoostingClassifier(
    random_state=RANDOM_STATE, n_estimators=20, learning_rate=0.05, max_features=2, max_depth=2)
cls3 = RandomForestClassifier(
    criterion="gini", max_depth=8, n_jobs=-1, random_state=RANDOM_STATE)

kfold = KFold(n_splits=6, shuffle=True, random_state=RANDOM_STATE)

scls = StackingClassifier(estimators=[('mlp', cls1), ('gbc', cls2), (
    'rfc', cls3)], stack_method='auto', n_jobs=-1, passthrough=False)

scores = {'accuracy': make_scorer(
    accuracy_score), 'f1_score': make_scorer(f1_score)}

results = cross_validate(scls, X, y, cv=kfold, n_jobs=-1, scoring=scores)

avg_fmeasure = np.mean(results['test_f1_score'])
avg_accuracy = np.mean(results['test_accuracy'])
# END CODE HERE


In [10]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('mlp',
                                MLPClassifier(max_iter=2000, random_state=42)),
                               ('gbc',
                                GradientBoostingClassifier(learning_rate=0.05,
                                                           max_depth=2,
                                                           max_features=2,
                                                           n_estimators=20,
                                                           random_state=42)),
                               ('rfc',
                                RandomForestClassifier(max_depth=8, n_jobs=-1,
                                                       random_state=42))],
                   n_jobs=-1)
F1 Weighted Score: 0.8763 & Balanced Accuracy: 0.853


You should achieve above 85% in both

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [11]:
# BEGIN CODE HERE
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier

ens1 = AdaBoostClassifier(random_state=RANDOM_STATE)
ens2 = RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE)
ens3 = BaggingClassifier(n_jobs=-1, random_state=RANDOM_STATE)
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

f_measures = dict()
accuracies = dict()

scores = {'accuracy': make_scorer(
    accuracy_score), 'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=7, random_state=RANDOM_STATE, shuffle=True)

results = cross_validate(tree, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["dt"] = results['test_f1_score']
accuracies["dt"] = results['test_accuracy']

results = cross_validate(ens1, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["rf"] = results['test_f1_score']
accuracies["rf"] = results['test_accuracy']

results = cross_validate(ens2, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["bag"] = results['test_f1_score']
accuracies["bag"] = results['test_accuracy']

results = cross_validate(ens3, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["ada"] = results['test_f1_score']
accuracies["ada"] = results['test_accuracy']
# END CODE HERE


In [12]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,np.round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,np.round(score,4)))

AdaBoostClassifier(random_state=42)
RandomForestClassifier(n_jobs=-1, random_state=42)
BaggingClassifier(n_jobs=-1, random_state=42)
DecisionTreeClassifier(random_state=42)
Classifier:dt -  F1 Weighted:[0.7532 0.7301 0.7535 0.745  0.7297 0.7756 0.7607]
Classifier:rf -  F1 Weighted:[0.8104 0.8176 0.8459 0.8261 0.8258 0.8516 0.8028]
Classifier:bag -  F1 Weighted:[0.8379 0.8342 0.8491 0.8271 0.8434 0.8617 0.8415]
Classifier:ada -  F1 Weighted:[0.7835 0.7921 0.8226 0.8083 0.8011 0.8232 0.7888]
Classifier:dt -  BalancedAccuracy:[0.715  0.6833 0.7033 0.7028 0.6895 0.7295 0.7195]
Classifier:rf -  BalancedAccuracy:[0.7817 0.7783 0.81   0.7997 0.788  0.8197 0.7663]
Classifier:bag -  BalancedAccuracy:[0.8033 0.79   0.8117 0.7913 0.8047 0.8247 0.8063]
Classifier:ada -  BalancedAccuracy:[0.755  0.7533 0.785  0.783  0.7663 0.7913 0.7613]


**2.2** Describe your classifiers and your results.

For this example I'm using three ensembles, created using `AdaBoostClassifier`, `RandomForestClassifier` and `BaggingClassifier`, each using `sklearn` default parameters apart from the `random_state` parameter.

This setup has proved that every ensemble is better than a simple `DecisionTreeClassifier` across both f1-scores and balanced-accuracy.

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

The `BaggingClassifier` is a process that can be very easily parallelized to increase performance. This is due to the fact that we're using the same training set as a starting point and doing work with the data itself. In this example a speedup is as easy as passing the parameter `n_jobs=-1` to the classifier.

In the case of a boosting classifier the above is not feasible. This is due to the fact that a boosting classifier's model is doing work based on the error of the model that came before it, thus a sequential order must be followed. The task is not able to be parallelized.

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [28]:
# BEGIN CODE HERE
from sklearn.svm import SVC, NuSVC

cls1 = SVC(random_state=RANDOM_STATE)

cls2 = MLPClassifier(random_state=RANDOM_STATE)

cls3 = NuSVC(random_state=RANDOM_STATE)

cls4 = RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE)

best_cls = StackingClassifier(estimators=[
                              cls1, cls2, cls3, cls4],  stack_method='auto', n_jobs=-1, passthrough=False)

scores = {'accuracy': make_scorer(
    accuracy_score), 'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

results = cross_validate(scls, X, y, cv=kfold, n_jobs=-1, scoring=scoring)

best_fmeasure = results["test_f1_score"].max()
best_accuracy = results["test_accuracy"].max()
# END CODE HERE


In [29]:
print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score:0.9026217228464418 & Balanced Accuracy:0.8806682577565632


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

I started by researching ensembles of classifiers on the internet, then through trial and error, I found the best order for the classifiers to be used. I left the parameters at the (sane) default values since trials are extremely comptutationally expensive using ensembles of 4 classifiers.

The final combination is `SVC`, `MLPClassifier`, `NuSVC` and a `RandomForestClassifier` combined using a `StackingClassifier` and it yielded a result of
`F1 Weighted-Score:0.90 & Balanced Accuracy:0.88`

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [1]:
# BEGIN CODE HERE
cls1 = SVC(random_state=RANDOM_STATE)
cls2 = MLPClassifier(random_state=RANDOM_STATE)
cls3 = NuSVC(random_state=RANDOM_STATE)
cls4 = RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE)
stacked_cls = StackingClassifier(estimators=[
    cls1, cls2, cls3, cls4],  stack_method='auto', n_jobs=-1, passthrough=False)
stacked_cls.fit(X, y)
# END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = stacked_cls.predict(test_set)

NameError: name 'SVC' is not defined

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [16]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!